# Relevancy Training to improve results of discovery

In [3]:
import os
import sys
from watson_developer_cloud import DiscoveryV1
from dotenv import load_dotenv, find_dotenv
import json
import csv
import requests

In [4]:
try:
    load_dotenv(find_dotenv())
except IOError:
    print('warning: no .env file loaded')

In [7]:
discovery = DiscoveryV1(
              #url=os.getenv('DISCOVERY_URL'),
              username=os.getenv('DISCOVERY_USERNAME'),
              password=os.getenv('DISCOVERY_PASSWORD'),
              version="2018-03-05"
            )
environment_id = os.getenv('DISCOVERY_ENVIRONMENT_ID')
configuration_id = os.getenv('DISCOVERY_CONFIGURATION_ID')
collection_id =  os.getenv('DISCOVERY_COLLECTION_ID')
# collection_id = '6cd34257-ac8c-43d5-b62f-61e0e9732eca'
print(' environment_id :',environment_id,'\n configuration_id : ',configuration_id,'\n collection_id : ',collection_id)                                 

 environment_id : fc71a276-5cb9-46a9-b24f-242588206648 
 configuration_id :  7b5331b5-7ecc-41dd-8646-bb532dc6bccb 
 collection_id :  5c8cc9b3-e8b7-409c-972c-053bf82e347c


In [1]:
#params = {}
#params["query"] = "%s" % ('What happens if you miss doses')
#discovery.query(environment_id=environment_id, collection_id=collection_id, count=1, natural_language_query='What measures should we take post floods?')

# Collect questions and get the response from discovery without relevancy training

In [9]:
with open ("/Users/rajeshgudikoti/Documents/rajesh/IBMDigital/Projects_docs/Indegene/questionair.txt") as questions:
    #open an output file to place the responses 
    filestr = "/Users/rajeshgudikoti/Documents/rajesh/IBMDigital/Projects_docs/Indegene/response-indegene-copy.csv"
    of = open(filestr, "w")
    writer = csv.writer(of, delimiter=",")
    
    #go through each question in file and prepare Discovery query paramaters 
    for line in questions:
        question = line.replace("\n", "")
        params = {}
        params["query"] = "%s" % (question)
        print(question,' \t***\t ',params["query"])
        #params["return"] = {"_id","body","title"} #these fields may need to be updated depending on the content being used 
        #params["return"] = "_id" #these fields may need to be updated depending on the content being used 
        key = "return"
        params.setdefault(key, [])
        #a[key].append(1)
        params["return"].append("_id")
        params["return"].append("body")
        params["return"].append("title")
        params["count"] = 4 
        #print('params["return"] ',params["return"])
        #run Discovery query to get results from untrained service 
        result = discovery.query(environment_id=environment_id, collection_id=collection_id, query_options=params)
        
        #create a row for each query and results 
        result_list = [question.encode("utf8")]
        for resultDoc in result["results"]:
            #print('resultDoc_id',resultDoc['id'])
            
            #print("resultDoc.get('extracted_metadata').get('title') " ,resultDoc.get('extracted_metadata').get('title'))
            id = resultDoc["id"]
            print('\n resultDoc.extracted_metadata \n', resultDoc.get('extracted_metadata'))
            #if('extracted_metadata'["title"] in resultDoc):
            #if(resultDoc.get('extracted_metadata').get('title') is not 'None' or 'no title'):
            if('title' in resultDoc.get('extracted_metadata')):
                print('title --> ', resultDoc['extracted_metadata']['title'])
                title = resultDoc['extracted_metadata']['title'].encode("utf8")
            else:
                title = 'title not present'
            #if(resultDoc.get('html') is not 'None'):
            if('html' in resultDoc):
                body = resultDoc['html'].encode("utf8")
                print('html present in resultDoc \n-->')
            if('text' in resultDoc):
                body = resultDoc['text'].encode("utf8")
                print('text present in resultDoc \n-->')
            else:
                body ='body not present'
                print('body not present in resultDoc \n-->' , resultDoc)
            
            print('id \n', id, '\n title \n',title)
            
            #result_list.extend([id,title,body,' ']) #leave a space to enter a relevance label for each doc 
            result_list.extend([id,title,' '])
            
        #write the row to the file 
        writer.writerow(result_list)
    
    of.close()
    #print('result_list',result_list)

What happens if you miss doses  	***	  What happens if you miss doses

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.doc


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html

...


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html


 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': 'c600f43e7a34da25b83d1eb96714d3fd5443c965', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_clopidogrel.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 d6aae77cbbd4ef194872f4b71497e6a2 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-18', 'sha1': '69559190b539b6cd1e2ecdce0390f606f6ffd3b7', 'author': 'Janhavi Dabir', 'filename': 'Discontinuation for surgery.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html present in resultDoc 
-->
text present in resultDoc 
-->
id 
 4523137ed853c344d62b8ab4de81ec1e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2017-12-22', 'sha1': '0e5553ef53cda01d9e5f8079722a72ee61c69502', 'author': 'Sandeep Gantotti', 'filename': 'DrugX_DI_ASA.docx', 'file_type': 'word', 'title': 'no title'}
title -->  no title
html

# Upload Training Data

In [10]:
#function for posting to training data endpoint 
def training_post(discovery_training_path, training_obj):
    #training_json = json.dumps(training_obj)
    training_json = training_obj
    headers = {
        "content-type": "application/json",
        "Accept" : "text/plain"
        }
    auth = (os.getenv('DISCOVERY_USERNAME'), os.getenv('DISCOVERY_PASSWORD'))
    
    response = requests.request(method="POST",url=discovery_training_path,data=training_json,headers=headers,auth=auth)
    #print(response.status_code , response.reason, '\n', response.request, response.url,response.text)
    print('\n\n',response.reason,response.text,'\n\n')
   

In [11]:
#open the training file and create new training data objects
with open('/Users/rajeshgudikoti/Documents/rajesh/IBMDigital/Projects_docs/Indegene/response-indegene-copy.csv','r') as training_doc:
    training_csv = csv.reader(training_doc, delimiter=',')    
    training_obj = {}
    training_obj["examples"] = []
    
    discovery_path = "https://gateway.watsonplatform.net/discovery/api/v1/environments/" + environment_id + "/collections/" + collection_id 
    discovery_training_path = discovery_path + "/training_data?version=2016-11-07"
    
    for row in training_csv:
        training_obj["natural_language_query"] = row[0]
        i = 1 
        print('\nrow ', row[0])
        training_obj["examples"]=[]
        #training_obj.fromkeys(["examples"])
        print('************\n traning example before loop*******', training_obj["examples"])
        for j in range(1,10):
            example_obj = {}
            
            #example_obj["relevance"] = row[i+3]
            # becuase we do not include body in result...see above cell
            example_obj["relevance"] = row[i+2]
            example_obj["document_id"] = row[i]
            
            print('example_obj.relevance', example_obj["relevance"])
            if (example_obj["relevance"]) is not ' ' or (example_obj["relevance"]) == 0:
                training_obj["examples"].append(example_obj)
            else:
                print('example_obj.relevance is zero or not specified')
                example_obj["relevance"] = 0
            #if (example_obj["relevance"]) != "0":
                #training_obj["examples"].append(example_obj)
            #else:
                #print('example_obj.relevance is zero')
            
            #training_obj["examples"].append(example_obj)
            
            #print('example_obj', example_obj)
            #training_obj["examples"] = example_obj
            i = i + 3
            
            print('\n\nrelevance object ',example_obj["relevance"] )
            #print('document id ', example_obj["document_id"])
            #print('traning example', training_obj["examples"])
            print('***** training_obj', json.dumps(training_obj))
        
            #send the training data to the discovery service 
        response = training_post(discovery_training_path, json.dumps(training_obj))
            #print(response.status_code , response.reason, '\n', response.request, response.url,response.text)
            
            #print('traning example', training_obj["examples"])


row  Natural Language Query
************
 traning example before loop******* []
example_obj.relevance Relevancy


relevance object  Relevancy
***** training_obj {"examples": [{"relevance": "Relevancy", "document_id": "Document_ID"}], "natural_language_query": "Natural Language Query"}
example_obj.relevance Relevancy


relevance object  Relevancy
***** training_obj {"examples": [{"relevance": "Relevancy", "document_id": "Document_ID"}, {"relevance": "Relevancy", "document_id": "Document_ID"}], "natural_language_query": "Natural Language Query"}
example_obj.relevance Relevancy


relevance object  Relevancy
***** training_obj {"examples": [{"relevance": "Relevancy", "document_id": "Document_ID"}, {"relevance": "Relevancy", "document_id": "Document_ID"}, {"relevance": "Relevancy", "document_id": "Document_ID"}], "natural_language_query": "Natural Language Query"}
example_obj.relevance Relevancy


relevance object  Relevancy
***** training_obj {"examples": [{"relevance": "Relevancy", "docu



 Created {"natural_language_query":"Is it okay to miss a dose?'","filter":"","examples":[{"document_id":"c95997bd22352b12be390d1da577f9f4","cross_reference":"","relevance":5,"created":"2018-08-23T17:17:58.693Z","updated":"2018-08-23T17:17:58.756Z"}],"query_id":"44a7fd9f1c9e2b1bea2337c2a17927f92bd9573c","created":"2018-08-23T17:17:58.718Z","updated":"2018-08-23T17:17:58.718Z"} 



row   If someone is prescribed DrugX to be takent twice daily, and they miss a dose, should they double up that evening?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someone is prescribed DrugX to be takent twice daily, and they miss a dose, should they double up that evening?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someon



 Created {"natural_language_query":"What happens if I miss a dose'","filter":"","examples":[],"query_id":"d7db569ea75e4ed01fc410fb98b94d8cb2d98924","created":"2018-08-23T17:18:04.137Z","updated":"2018-08-23T17:18:04.137Z"} 



row   If someone taking DrugX missed their dose yesterday what should they do?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someone taking DrugX missed their dose yesterday what should they do?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someone taking DrugX missed their dose yesterday what should they do?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someone takin



 Created {"natural_language_query":"When is DrugX recommended to be stopped prior to a surgery?'","filter":"","examples":[{"document_id":"4523137ed853c344d62b8ab4de81ec1e","cross_reference":"","relevance":5,"created":"2018-08-23T17:18:09.189Z","updated":"2018-08-23T17:18:09.251Z"}],"query_id":"1445fae3b026fcd2b6437dc30656538aa1c05ef3","created":"2018-08-23T17:18:09.216Z","updated":"2018-08-23T17:18:09.216Z"} 



row   When should product be discontinued prior to surgery? ( surgery is not prodcut-related)'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " When should product be discontinued prior to surgery? ( surgery is not prodcut-related)'"}
example_obj.relevance 5


relevance object  5
***** training_obj {"examples": [{"relevance": "5", "document_id": "4523137ed853c344d62b8ab4de81ec1e"}], "natural_language_query": " When sho



 Created {"natural_language_query":"Patient on DrugX who is having a scheduled dental procedure-tooth extraction unrelated to DrugX.  What protocol does DrugX follow with discontinuing prior to dental procedure or surgery?'","filter":"","examples":[{"document_id":"4523137ed853c344d62b8ab4de81ec1e","cross_reference":"","relevance":5,"created":"2018-08-23T17:18:14.347Z","updated":"2018-08-23T17:18:14.385Z"}],"query_id":"dfc0bb9f370bc0129b742d1bd8163f97cf197dd9","created":"2018-08-23T17:18:14.351Z","updated":"2018-08-23T17:18:14.351Z"} 



row   Does DrugX need to be held for routine dental cleaning?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " Does DrugX need to be held for routine dental cleaning?'"}
example_obj.relevance 5


relevance object  5
***** training_obj {"examples": [{"relevance": "5", "document_id": "4523137ed8

...



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'What do I do if I miss a dose?'||' id ab40183a46ad49a9af13bda7d7d47e95005c48b already exists in collection 5c8cc9b3-e8b7-409c-972c-053bf82e347c, environment fc71a276-5cb9-46a9-b24f-242588206648","code":409} 



row   What is the information available on DrugX on a possible missed dose?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What is the information available on DrugX on a possible missed dose?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What is the information available on DrugX on a possible missed dose?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "nat



 Created {"natural_language_query":"What is the recommendation if a dose is missed?'","filter":"","examples":[{"document_id":"c95997bd22352b12be390d1da577f9f4","cross_reference":"","relevance":5,"created":"2018-08-23T17:20:54.942Z","updated":"2018-08-23T17:20:54.983Z"}],"query_id":"cc91c8cc25395e9723e74177a3d52633a57506c9","created":"2018-08-23T17:20:54.953Z","updated":"2018-08-23T17:20:54.953Z"} 



row   What do I do for a missed dose?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What do I do for a missed dose?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What do I do for a missed dose?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examp



 Created {"natural_language_query":"What should one do if a dose of DrugX is missed?'","filter":"","examples":[{"document_id":"c95997bd22352b12be390d1da577f9f4","cross_reference":"","relevance":5,"created":"2018-08-23T17:21:01.971Z","updated":"2018-08-23T17:21:02.026Z"}],"query_id":"a4362ba9ad98a045f9c523d44a04c3e60231dee8","created":"2018-08-23T17:21:01.996Z","updated":"2018-08-23T17:21:01.996Z"} 



row   If someone were to miss a dose of DrugX, what is the recommendation?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someone were to miss a dose of DrugX, what is the recommendation?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " If someone were to miss a dose of DrugX, what is the recommendation?'"}
example



 Created {"natural_language_query":"What should I do if I miss a dose of DrugX?'","filter":"","examples":[{"document_id":"c95997bd22352b12be390d1da577f9f4","cross_reference":"","relevance":5,"created":"2018-08-23T17:21:07.138Z","updated":"2018-08-23T17:21:07.196Z"}],"query_id":"ee6f206f3bd42cd71cf20bc41b90f891ace8b80a","created":"2018-08-23T17:21:07.164Z","updated":"2018-08-23T17:21:07.164Z"} 



row   What information is there for a possible missed dose on DrugX?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What information is there for a possible missed dose on DrugX?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What information is there for a possible missed dose on DrugX?'"}
example_obj.relevance  
exampl



 Created {"natural_language_query":"If someone forgot to take DrugX in the evening, took the medication the next morning when they remembered, when should the next dose be taken?'","filter":"","examples":[{"document_id":"c95997bd22352b12be390d1da577f9f4","cross_reference":"","relevance":5,"created":"2018-08-23T17:21:12.207Z","updated":"2018-08-23T17:21:12.240Z"}],"query_id":"aa6c4328187bf7ab418635aabfbecb82c483662a","created":"2018-08-23T17:21:12.210Z","updated":"2018-08-23T17:21:12.210Z"} 



row   What should someone do if they forget to take their DrugX?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What should someone do if they forget to take their DrugX?'"}
example_obj.relevance  
example_obj.relevance is zero or not specified


relevance object  0
***** training_obj {"examples": [], "natural_language_query": " What 

# Query post relevance training

In [12]:
discovery.query(environment_id, collection_id, filter=None, query=None, natural_language_query='What happens if you miss doses', 
                passages=True, 
      aggregation=None, count=None, return_fields=None, offset=None, sort=None, highlight=None, 
      passages_fields=None, passages_count=None, passages_characters=None, deduplicate=None, deduplicate_field=None)


{'matching_results': 8,
 'passages': [{'document_id': 'c95997bd22352b12be390d1da577f9f4',
   'end_offset': 552,
   'field': 'text',
   'passage_score': 8.69685660825127,
   'passage_text': 'The patient should continue with the regular 15 mg twice daily intake as recommended on the following day.\n\nFor patients receiving 20 mg, 15 mg or 10 mg once daily: The patient should take the missed Xarelto® dose immediately.',
   'start_offset': 326},
  {'document_id': 'c95997bd22352b12be390d1da577f9f4',
   'end_offset': 964,
   'field': 'html',
   'passage_score': 8.544146220046683,
   'passage_text': 'day.</p>\n<p dir="ltr">For patients receiving 20 mg, 15 mg or 10 mg once daily: The patient should take the missed Xarelto® dose immediately.</p>\n</div>\n\n</body></html>',
   'start_offset': 797},
  {'document_id': 'd6aae77cbbd4ef194872f4b71497e6a2',
   'end_offset': 796,
   'field': 'html',
   'passage_score': 4.657042810830453,
   'passage_text': ', however, clopidogrel (300 mg loading dose f


# Delete All Training Data

In [8]:
discovery.delete_all_training_data(environment_id, collection_id)

# Add training data + examples for individual NLQ

In [13]:
data = {
    "natural_language_query": "New NLQ", 
        
    "examples": [{"document_id": "493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0", "relevance": 2}, 
                 {"document_id": "18ed484753a7f9845b364bdf74f2eb79", "relevance": 1}]
}

headers = {
        "content-type": "application/json",
        "Accept" : "text/plain"
        }

discovery_path = "https://gateway.watsonplatform.net/discovery/api/v1/environments/" + environment_id + "/collections/" + collection_id + "/training_data"
discovery_training_path = discovery_path + "?version=2018-03-05"

username = os.getenv('DISCOVERY_USERNAME')
password = os.getenv('DISCOVERY_PASSWORD')

auth = (username, password)

print(discovery_path, '\n', discovery_training_path)
print(data)
#data = json.dumps(data)
#print(data)

response = requests.request(method="POST",url=discovery_training_path,json=data,headers=headers,auth=auth)
#response = requests.post(url=discovery_path,params=params,json=data,headers=headers)
#print(response.json())

#add_training_data(discovery, environment_id, collection_id, natural_language_query='what are dos and donts during earthquake', filter=None, 
                  #examples=examples, **kwargs)
print(response.status_code , response.reason, '\n', response.request, response.url,response.text)

https://gateway.watsonplatform.net/discovery/api/v1/environments/fc71a276-5cb9-46a9-b24f-242588206648/collections/5c8cc9b3-e8b7-409c-972c-053bf82e347c/training_data 
 https://gateway.watsonplatform.net/discovery/api/v1/environments/fc71a276-5cb9-46a9-b24f-242588206648/collections/5c8cc9b3-e8b7-409c-972c-053bf82e347c/training_data?version=2018-03-05
{'natural_language_query': 'New NLQ', 'examples': [{'document_id': '493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0', 'relevance': 2}, {'document_id': '18ed484753a7f9845b364bdf74f2eb79', 'relevance': 1}]}
201 Created 
 <PreparedRequest [POST]> https://gateway.watsonplatform.net/discovery/api/v1/environments/fc71a276-5cb9-46a9-b24f-242588206648/collections/5c8cc9b3-e8b7-409c-972c-053bf82e347c/training_data?version=2018-03-05 {"natural_language_query":"New NLQ","filter":"","examples":[{"document_id":"493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0","cross_reference":"","relevance":2,"created":"2018-08-23T17:24:0

In [13]:
json_output = json.loads(response.text)
query_id=json_output['query_id']
print(query_id)

8bd44b5c90fafd50a7cff98618c2837211c6c79f


# Add example to existing query

In [17]:
query_id = 8bd44b5c90fafd50a7cff98618c2837211c6c79f
data = {
   "document_id": "493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d06",
   "relevance": 1
 }
json_output = json.loads(response.text)
add_example_path = discovery_path + "/" + query_id + "/examples?version=2018-03-05"
print(add_example_path)
response = requests.request(method="POST",url=add_example_path,json=data,headers=headers,auth=auth)
print(response.status_code , response.reason, '\n', response.request, response.url,response.text)

https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data/8bd44b5c90fafd50a7cff98618c2837211c6c79f/examples?version=2018-03-05
201 Created 
 <PreparedRequest [POST]> https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data/8bd44b5c90fafd50a7cff98618c2837211c6c79f/examples?version=2018-03-05 {"document_id":"493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d06","cross_reference":"","relevance":1,"created":"2018-08-08T05:23:54.384Z","updated":"2018-08-08T05:23:54.530Z"}
